In [ ]:
import numpy as np
import cv2

import PIL.Image as Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir=tf.keras.utils.get_file('flower_photos',origin=dataset_url,cache_dir='.',untar=True)#cache_dir->choosing directory, untar=unzipping

228813984/228813984 [==============================] - 2s 0us/step


In [ ]:
import pathlib
data_dir=pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [ ]:
list(data_dir.glob('*/*.jpg'))

[PosixPath('datasets/flower_photos/sunflowers/15072973261_73e2912ef2_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/5973935729_2868f2db1f_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/15030133005_9728102622_z.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/14646283472_50a3ae1395.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/9555824387_32b151e9b0_m.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/14698136411_23bdcff7bf_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/4869189730_f47c124cda_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/3594967811_697184b026_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/6042014768_b57f0bfc79_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/3334350831_f8755a2095_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/15054866898_60ee50ec6b_n.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/18972803569_1a0634f398_m.jpg'),
 PosixPath('datasets/flower_photos/sunflowers/5437996076_cf7e2ac32e_n.jp

In [ ]:
image_dir={
    'daisy':list(data_dir.glob('roses/*')),
    'dandelion':list(data_dir.glob('dandelion/*')),
    'roses':list(data_dir.glob('roses/*')),
    'tulips':list(data_dir.glob('tulips/*'))


}

In [ ]:
image_dir['roses']

[PosixPath('datasets/flower_photos/roses/1446097778_97149b8362.jpg'),
 PosixPath('datasets/flower_photos/roses/4243078361_7b92a932cd_n.jpg'),
 PosixPath('datasets/flower_photos/roses/15222804561_0fde5eb4ae_n.jpg'),
 PosixPath('datasets/flower_photos/roses/18741313803_1bbf842fc6_n.jpg'),
 PosixPath('datasets/flower_photos/roses/475936554_a2b38aaa8e.jpg'),
 PosixPath('datasets/flower_photos/roses/9353111163_7a89b2df35_n.jpg'),
 PosixPath('datasets/flower_photos/roses/5083072098_81587295d5.jpg'),
 PosixPath('datasets/flower_photos/roses/3265902330_d8b1e44545.jpg'),
 PosixPath('datasets/flower_photos/roses/3072908271_08764c732a_m.jpg'),
 PosixPath('datasets/flower_photos/roses/23232710191_cc57620cd5.jpg'),
 PosixPath('datasets/flower_photos/roses/15750320284_22ef21c682.jpg'),
 PosixPath('datasets/flower_photos/roses/3909587261_f8cd3e7fe7.jpg'),
 PosixPath('datasets/flower_photos/roses/6347846935_51e3dc2481_n.jpg'),
 PosixPath('datasets/flower_photos/roses/6108118824_5b0231a56d.jpg'),
 Posi

In [ ]:
label_dict={
    'daisy':0,
    'dandelion':1,
    'roses':2,
    'tulips':3

}

In [ ]:
img=cv2.imread(str(image_dir['roses'][0]))


In [ ]:
img.shape

(333, 500, 3)

In [ ]:
cv2.resize(img,IMAGE_SHAPE).shape

(224, 224, 3)

In [ ]:
x,y=[],[]

for flower_name,images in image_dir.items():
    for image in images:
        img=cv2.imread(str(image))
        resize_img=cv2.resize(img,(224,224))
        y.append(label_dict[flower_name])
        x.append(resize_img)



In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.3,random_state=10)

X_train=X_train/255
X_test=X_test/255


In [ ]:
#model prediction

predicted_array=classifier.predict(np.array(X_test))
predicted=[np.argmax(i) for i in predicted_array]

28/28 [==============================] - 28s 974ms/step


In [ ]:
predicted[:10]

[948, 585, 329, 947, 884, 109, 724, 10, 108, 912]

In [ ]:
google_data_url='https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
gdata_dir=tf.keras.utils.get_file('ImageNetLabels.txt',origin=google_data_url,cache_dir='.')

In [ ]:
image_label=[]

with open('/content/datasets/ImageNetLabels.txt','r') as f:
    image_label=f.read().splitlines()

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
num_of_flowers = 5


model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:

model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train, Y_train,batch_size=16, epochs=5)

Epoch 1/5
131/131 [==============================] - 71s 530ms/step - loss: 0.4421 - acc: 0.7645
Epoch 2/5
131/131 [==============================] - 67s 513ms/step - loss: 0.4031 - acc: 0.7770
Epoch 3/5
131/131 [==============================] - 69s 525ms/step - loss: 0.3831 - acc: 0.7966
Epoch 4/5
131/131 [==============================] - 68s 518ms/step - loss: 0.3741 - acc: 0.7847
Epoch 5/5
131/131 [==============================] - 69s 528ms/step - loss: 0.3616 - acc: 0.7986
